Biểu diễn vector thưa dùng danh sách liên kết. Vector thưa như văn bản được biểu diễn dưới dạng vector nhị phân (1/0). Nếu văn bản có chứa từ thì giá trị tương ứng sẽ là 1, ngược lại sẽ là 0. Từ điển sẽ là tập các văn bản (tự crawl) . Các phép toán cần hỗ trợ là gộp văn bản (cộng vector thưa), so sánh độ tương đồng của 2 văn bản (tìm giao của 2 vector thưa sau đó tính theo khoảng cách hamming).

## Phân tích & Đặt nền móng

### 1. Tại sao dùng Danh sách liên kết?

* Một "từ điển" (vocabulary) có thể có 100.000 từ.
* Một văn bản (document) trung bình chỉ chứa 300 từ.
* Nếu dùng `list` Python (array), mỗi văn bản sẽ là một `list` 100.000 phần tử, trong đó **99.700 phần tử là số 0**. Đây là một sự lãng phí bộ nhớ cực kỳ lớn.
* **Logic:** Chỉ cần lưu trữ những phần tử **khác 0**. Vì giá trị luôn là 1 (vector nhị phân), nên thực chất chỉ cần lưu **chỉ số (index)** của những từ có xuất hiện.
* Danh sách liên kết rất lý tưởng để lưu một tập hợp các "chỉ số" mà không cần biết trước số lượng.

### 2. Yêu cầu "Tối quan trọng" (Quy tắc vàng)

* Tất cả các phép toán (gộp, giao) đều yêu cầu so sánh chỉ số.
* Để so sánh hiệu quả, danh sách liên kết **BẮT BUỘC** phải được duy trì ở trạng thái **ĐÃ SẮP XẾP TĂNG DẦN** theo `index`.
* Ví dụ: một văn bản chứa từ ở index `[5, 50, 2]` phải được lưu trữ dưới dạng: `Node(2) -> Node(5) -> Node(50) -> None`.
* Đây là mấu chốt để mọi thuật toán bên dưới chạy nhanh.

---

## Bước 1: Tư duy về Cấu trúc dữ liệu

Cần 2 `class` để mô hình hóa bài toán:

### 1. `Node` (Mắt xích)

Đây là viên gạch cơ bản. Nó không cần lưu `data=1`, vì sự *tồn tại* của Node đã ngụ ý là 1.

* `index` (int): Chỉ số của từ trong từ điển.
* `next` (Node): Tham chiếu (con trỏ) đến từ tiếp theo có trong văn bản.

### 2. `SparseVector` (Vector thưa)

Đây là `class` quản lý toàn bộ văn bản (chuỗi các Node).

* `head` (Node): Tham chiếu đến `Node` đầu tiên của danh sách.
* Phương thức quan trọng nhất: `insert(index)`. Phương thức này phải đảm bảo "Quy tắc vàng" ở Bước 0: **chèn vào đúng vị trí để giữ danh sách được sắp xếp.**

#### Logic của `insert(index)`:

1.  Tạo `new_node = Node(index)`.
2.  **Trường hợp 1: Danh sách rỗng (`self.head is None`)**
    * `self.head = new_node`. Xong.
3.  **Trường hợp 2: Chèn vào đầu (`index < self.head.index`)**
    * `new_node.next = self.head`.
    * `self.head = new_node`. Xong.
4.  **Trường hợp 3: Chèn vào giữa hoặc cuối**
    * Dùng một con trỏ `current = self.head`.
    * Duyệt `while` (vòng lặp) cho đến khi tìm được nút `current` nằm *ngay trước* vị trí cần chèn. (Điều kiện dừng: `current.next is None` hoặc `current.next.index > index`).
    * Kiểm tra trùng lặp (nếu `current.index == index` hoặc `current.next.index == index`). Nếu trùng, không làm gì cả.
    * **Thực hiện chèn:**
        * `new_node.next = current.next`
        * `current.next = new_node`. Xong.

---

## Bước 2: Tư duy Phép toán 1 - Gộp văn bản (Phép CỘNG Vector)

Phép toán này chính là **Phép HỢP (Union)** của hai tập hợp.

* **Input:** `vec1` (đã sắp xếp), `vec2` (đã sắp xếp).
* **Output:** `vec_result` (một `SparseVector` mới, cũng phải được sắp xếp).
* **Logic:** Sử dụng thuật toán "Trộn" (Merge) tương tự như trong **Merge Sort**.

#### Tư duy thuật toán `merge(vec1, vec2)`:

1.  Tạo một `vec_result = SparseVector()` để chứa kết quả.
2.  Tạo 2 con trỏ: `p1 = vec1.head`, `p2 = vec2.head`.
3.  Dùng một con trỏ `tail` cho `vec_result` để tối ưu việc chèn vào cuối (O(1)).
4.  `while p1 is not None HOẶC p2 is not None:` (lặp cho đến khi duyệt hết cả hai)
    * **TH 1: `p1` hết, chỉ còn `p2`**
        * Lấy `index` từ `p2`. Thêm vào `vec_result`.
        * `p2 = p2.next`.
    * **TH 2: `p2` hết, chỉ còn `p1`**
        * Lấy `index` từ `p1`. Thêm vào `vec_result`.
        * `p1 = p1.next`.
    * **TH 3: Cả hai đều còn. So sánh:**
        * **Nếu `p1.index < p2.index`:**
            * Lấy `index` từ `p1`. Thêm vào `vec_result`.
            * `p1 = p1.next`.
        * **Nếu `p2.index < p1.index`:**
            * Lấy `index` từ `p2`. Thêm vào `vec_result`.
            * `p2 = p2.next`.
        * **Nếu `p1.index == p2.index` (trùng nhau):**
            * Lấy `index` từ `p1` (hoặc `p2`). Thêm 1 lần duy nhất vào `vec_result`.
            * `p1 = p1.next`.
            * `p2 = p2.next`.
5.  Trả về `vec_result`.

Thuật toán này có độ phức tạp **O(M + N)**, với M và N là số từ trong 2 văn bản (cực kỳ hiệu quả).

---

## Bước 3: Tư duy Phép toán 2 - So sánh (Giao & Hamming)

Bài toán yêu cầu 2 bước:
1.  Tìm **GIAO (Intersection)** của 2 vector.
2.  Tính **Khoảng cách Hamming**.

### A. Tìm Giao (Intersection)

* **Logic:** Gần giống hệt phép `merge` ở trên, nhưng chỉ xử lý khi cả hai con trỏ CÙNG tồn tại và chỉ lấy giá trị khi chúng BẰNG NHAU.
* **Tư duy thuật toán `intersect(vec1, vec2)`:**
    1.  Tạo `vec_result = SparseVector()`.
    2.  Tạo 2 con trỏ: `p1 = vec1.head`, `p2 = vec2.head`.
    3.  `while p1 is not None VÀ p2 is not None:` (lặp khi cả hai CÙNG còn)
        * **Nếu `p1.index < p2.index`:** (p1 nhỏ hơn, p1 không thể là giao)
            * `p1 = p1.next`.
        * **Nếu `p2.index < p1.index`:** (p2 nhỏ hơn, p2 không thể là giao)
            * `p2 = p2.next`.
        * **Nếu `p1.index == p2.index` (Tìm thấy GIAO!):**
            * Lấy `index` này. Thêm vào `vec_result`.
            * `p1 = p1.next`.
            * `p2 = p2.next`.
    4.  Trả về `vec_result`. (Đây là vector chứa các từ chung).

### B. Tính Khoảng cách Hamming

* **Định nghĩa:** Khoảng cách Hamming là "số vị trí mà 2 vector có giá trị khác nhau".
* **Phân tích 4 trường hợp tại một `index` bất kỳ:**
    1.  `vec1[i] = 0`, `vec2[i] = 0` -> Giống nhau (Match).
    2.  `vec1[i] = 1`, `vec2[i] = 1` -> Giống nhau (Match).
    3.  `vec1[i] = 1`, `vec2[i] = 0` -> **Khác nhau (Mismatch)**.
    4.  `vec1[i] = 0`, `vec2[i] = 1` -> **Khác nhau (Mismatch)**.
* **Logic:** Khoảng cách Hamming = (Số lượng Mismatch 3) + (Số lượng Mismatch 4).
    * "Mismatch 3" là các `index` **chỉ có trong `vec1`**.
    * "Mismatch 4" là các `index` **chỉ có trong `vec2`**.
* Đây chính là **Phép HIỆU ĐỐI XỨNG (Symmetric Difference)**.
* **Công thức tính nhanh (Tối ưu):**
    * `|vec1|` = Số nút trong `vec1` (tổng số từ của nó).
    * `|vec2|` = Số nút trong `vec2`.
    * `|Intersection|` = Số nút trong vector GIAO (tính ở bước A).
    * Số lượng Mismatch 3 = `|vec1| - |Intersection|`
    * Số lượng Mismatch 4 = `|vec2| - |Intersection|`
    * **Công thức cuối cùng:**
        `Hamming_Distance = (|vec1| - |Intersection|) + (|vec2| - |Intersection|)`
        **`Hamming_Distance = |vec1| + |vec2| - 2 * |Intersection|`**
* **Tối ưu code:** Không cần *tạo* ra `vec_result` của phép Giao, mà chỉ cần *đếm* số lượng phần tử giao (`get_intersection_size`) rồi áp dụng công thức.

In [1]:
import sys

# Tăng giới hạn đệ quy cho các thao tác trên danh sách liên kết dài (nếu cần)
sys.setrecursionlimit(2000)

## --------------------------------------------------
## BƯỚC 1: CẤU TRÚC DỮ LIỆU CƠ BẢN
## --------------------------------------------------

class Node:
    """
    Viên gạch cơ bản: Một 'Nút' (mắt xích).
    Nó chỉ cần lưu 'index' của từ và tham chiếu 'next'.
    """
    def __init__(self, index):
        self.index = index  # Chỉ số của từ trong từ điển
        self.next = None    # Nút tiếp theo (Node)

class SparseVector:
    """
    Lớp đại diện cho một Vector thưa (một văn bản).
    Quản lý một danh sách liên kết các Nút, đã được SẮP XẾP.
    """
    def __init__(self):
        self.head = None
        self.length = 0  # Theo dõi độ dài để tính toán O(1)

    def get_length(self):
        """Trả về số lượng từ (phần tử khác 0) trong vector."""
        return self.length

    def insert(self, index):
        """
        Chèn một 'index' vào vector, đảm bảo duy trì thứ tự sắp xếp.
        Nếu index đã tồn tại, bỏ qua.
        """
        new_node = Node(index)

        # 1. Trường hợp: Danh sách rỗng
        if self.head is None:
            self.head = new_node
            self.length += 1
            return

        # 2. Trường hợp: Chèn vào đầu
        if index < self.head.index:
            new_node.next = self.head
            self.head = new_node
            self.length += 1
            return
        
        # 3. Trường hợp: Chèn vào giữa hoặc cuối
        current = self.head
        while current.next is not None and current.next.index < index:
            current = current.next

        # 4. Kiểm tra trùng lặp (với current hoặc current.next)
        if current.index == index or (current.next and current.next.index == index):
            # Đã tồn tại, không làm gì cả
            return

        # 5. Chèn vào sau 'current'
        new_node.next = current.next
        current.next = new_node
        self.length += 1

    def print_vector(self):
        """Hàm tiện ích để in vector ra màn hình."""
        nodes = []
        current = self.head
        while current:
            nodes.append(str(current.index))
            current = current.next
        print(" -> ".join(nodes) + " -> None")

## --------------------------------------------------
## BƯỚC 2: PHÉP TOÁN GỘP VĂN BẢN (MERGE/UNION)
## --------------------------------------------------

def merge_vectors(vec1, vec2):
    """
    Gộp (HỢP/UNION) hai vector thưa.
    Tạo ra một vector mới chứa tất cả các chỉ số từ cả hai vector.
    """
    # Dùng một con trỏ 'dummy_head' để đơn giản hóa logic chèn
    dummy_head = Node(0) 
    tail = dummy_head  # 'tail' là Nút cuối cùng của danh sách kết quả
    
    p1 = vec1.head
    p2 = vec2.head
    
    while p1 is not None or p2 is not None:
        if p1 is None:
            # p1 hết, chỉ còn p2
            tail.next = Node(p2.index)
            p2 = p2.next
        elif p2 is None:
            # p2 hết, chỉ còn p1
            tail.next = Node(p1.index)
            p1 = p1.next
        elif p1.index < p2.index:
            # Lấy từ p1
            tail.next = Node(p1.index)
            p1 = p1.next
        elif p2.index < p1.index:
            # Lấy từ p2
            tail.next = Node(p2.index)
            p2 = p2.next
        else: # p1.index == p2.index
            # Trùng nhau, lấy 1 lần
            tail.next = Node(p1.index)
            p1 = p1.next
            p2 = p2.next
            
        # Di chuyển con trỏ 'tail' đến nút vừa thêm
        tail = tail.next

    # Tạo đối tượng SparseVector mới từ danh sách đã gộp
    result_vector = SparseVector()
    result_vector.head = dummy_head.next # Bỏ qua dummy_head
    
    # Tính toán lại độ dài (hoặc cập nhật trong vòng lặp)
    current = result_vector.head
    count = 0
    while current:
        count += 1
        current = current.next
    result_vector.length = count

    return result_vector

## --------------------------------------------------
## BƯỚC 3: PHÉP TOÁN SO SÁNH (GIAO & HAMMING)
## --------------------------------------------------

def _get_intersection_size(vec1, vec2):
    """
    Hàm trợ giúp (private): Đếm số lượng phần tử GIAO (intersection)
    giữa hai vector đã sắp xếp.
    """
    count = 0
    p1 = vec1.head
    p2 = vec2.head

    while p1 is not None and p2 is not None:
        if p1.index < p2.index:
            p1 = p1.next
        elif p2.index < p1.index:
            p2 = p2.next
        else: # p1.index == p2.index
            # Tìm thấy GIAO
            count += 1
            p1 = p1.next
            p2 = p2.next
    
    return count

def calculate_hamming_distance(vec1, vec2):
    """
    Tính khoảng cách Hamming giữa hai vector thưa.
    Công thức: |vec1| + |vec2| - 2 * |Giao(vec1, vec2)|
    """
    len1 = vec1.get_length()
    len2 = vec2.get_length()
    
    intersection_size = _get_intersection_size(vec1, vec2)
    
    # Khoảng cách Hamming = (len1 - intersection_size) + (len2 - intersection_size)
    distance = len1 + len2 - 2 * intersection_size
    
    return distance

## --------------------------------------------------
## VÍ DỤ SỬ DỤNG
## --------------------------------------------------

# Giả sử chúng ta có một từ điển:
# 0: "python", 1: "code", 2: "logic", 3: "vector", 4: "sparse", 5: "data"

print("--- Tạo Vector ---")
# Văn bản 1: "python logic sparse data"
doc1 = SparseVector()
doc1.insert(0) # "python"
doc1.insert(2) # "logic"
doc1.insert(4) # "sparse"
doc1.insert(5) # "data"
print("Văn bản 1:", end=" ")
doc1.print_vector()
print(f"Độ dài: {doc1.get_length()}")

# Văn bản 2: "python data vector"
doc2 = SparseVector()
doc2.insert(5) # "data"
doc2.insert(0) # "python"
doc2.insert(3) # "vector"
print("\nVăn bản 2:", end=" ")
doc2.print_vector()
print(f"Độ dài: {doc2.get_length()}")

print("\n--- Phép toán GỘP (Merge) ---")
merged_doc = merge_vectors(doc1, doc2)
print("Văn bản gộp:", end=" ")
merged_doc.print_vector()
print(f"Độ dài: {merged_doc.get_length()}")

print("\n--- Phép toán KHOẢNG CÁCH HAMMING ---")
# doc1: [1, 0, 1, 0, 1, 1]
# doc2: [1, 0, 0, 1, 0, 1]
# Khác nhau tại index 2, 3, 4.
# Khoảng cách Hamming = 3

# Tính theo công thức:
# |doc1| = 4
# |doc2| = 3
# Giao = {0, 5} -> Kích thước giao = 2
# Hamming = 4 + 3 - 2 * 2 = 7 - 4 = 3
distance = calculate_hamming_distance(doc1, doc2)
print(f"Khoảng cách Hamming giữa doc1 và doc2 là: {distance}")

--- Tạo Vector ---
Văn bản 1: 0 -> 2 -> 4 -> 5 -> None
Độ dài: 4

Văn bản 2: 0 -> 3 -> 5 -> None
Độ dài: 3

--- Phép toán GỘP (Merge) ---
Văn bản gộp: 0 -> 2 -> 3 -> 4 -> 5 -> None
Độ dài: 5

--- Phép toán KHOẢNG CÁCH HAMMING ---
Khoảng cách Hamming giữa doc1 và doc2 là: 3
